In [2]:
import torch
import torch.nn as nn
import torchvision.models as models

class VGGModel(nn.Module):
    def __init__(self):
        super(VGGModel, self).__init__()
        self.vgg16 = models.vgg16(pretrained=True)

        # replace output layer according to problem
        in_feats = self.vgg16.classifier[6].in_features
        self.vgg16.classifier[6] = nn.Linear(in_feats, 2)

    def forward(self, x):
        x = self.vgg16(x)
        return x

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize and load the model
model = VGGModel().to(device)
model.load_state_dict(torch.load('/home/Alzhimers/best_model_alzhimers.pth', map_location=device))
model.eval()  # Set the model to evaluation mode


VGGModel(
  (vgg16): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (16):

In [3]:
dummy_input = torch.randn(1, 3, 255, 255).to(device)

In [6]:
onnx_model_path = "vgg16_custom_alzhimer.onnx"

# Export the model
torch.onnx.export(
    model,                       # model being run
    dummy_input,                 # model input (or a tuple for multiple inputs)
    onnx_model_path,             # where to save the model (can be a file or file-like object)
    export_params=True,          # store the trained parameter weights inside the model file
    opset_version=11,            # the ONNX version to export the model to
    do_constant_folding=True,    # whether to execute constant folding for optimization
    input_names=['input'],       # the model's input names
    output_names=['output'],     # the model's output names
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}  # variable length axes
)

print(f"Model has been converted to ONNX and saved at {onnx_model_path}")

Model has been converted to ONNX and saved at vgg16_custom_alzhimer.onnx


In [12]:
import onnx

onnx_model_path='/home/Alzhimers/vgg16_custom_alzhimer.onnx'
onnx_model = onnx.load(onnx_model_path)


onnx.checker.check_model(onnx_model)

print("ONNX model is valid!")

ONNX model is valid!
